In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,PowerTransformer,StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

In [18]:
path='C:\\Users\\amanc\\Desktop\\python new project\\Machine Learning\\Credit Score Classification\\data\\train_new.csv'
data=pd.read_csv(path)

In [19]:
data.head()

,Unnamed: 0,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,...,Auto_Loan,Credit_Builder_Loan,Debt_Consolidation_Loan,Home_Equity_Loan,Mortgage_Loan,No_Loan,Not_Specified,Payday_Loan,Personal_Loan,Student_Loan
0,0,17,Lawyer,30625.94,2706.16,6,5,27.0,2,57,...,0,1,0,1,0,0,0,0,0,0
1,1,25,Mechanic,52312.68,4250.39,6,5,17.0,4,7,...,0,1,0,1,0,0,1,1,0,0
2,2,18,Media_Manager,113781.39,9549.78,1,4,1.0,0,13,...,0,0,0,0,0,1,0,0,0,0
3,3,43,Doctor,58918.47,5208.87,3,3,17.0,3,27,...,0,1,1,0,0,0,0,0,0,1
4,4,43,Mechanic,98620.98,7962.42,3,3,6.0,3,12,...,0,0,1,0,0,0,0,0,1,1


In [20]:
x_train,x_test,y_train,y_test=train_test_split(data.drop('Credit_Score',axis=1),data['Credit_Score'],test_size=0.2,random_state=42)

In [24]:
x_train.shape

(10000, 32)

In [21]:
le=LabelEncoder()
y_train_label=le.fit_transform(y_train)
y_test_label=le.transform(y_test)

In [42]:
encode=ColumnTransformer([
    ('ohe',OneHotEncoder(sparse_output=False,handle_unknown='error',drop='first'),['Occupation','Payment_of_Min_Amount','Payment_Behaviour']),
    ('oe',OrdinalEncoder(categories=[['Bad', 'Standard', 'Good']]),['Credit_Mix'])
],remainder='passthrough')

In [43]:
xx=encode.fit_transform(x_train)
xx.shape

(10000, 49)

In [44]:
scale=ColumnTransformer([
    ('scaler',StandardScaler(),slice(0,50))
],remainder='passthrough')

In [45]:
transform=ColumnTransformer([
    ('yeo',PowerTransformer(method='yeo-johnson'),slice(0,50))
],remainder='passthrough')

In [49]:
pipe=make_pipeline(encode,scale,transform)

In [50]:
x_train_tr=pipe.fit_transform(x_train)
x_test_tr=pipe.transform(x_test)

In [214]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier,VotingClassifier,RandomForestClassifier,GradientBoostingClassifier
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.decomposition import PCA
import xgboost as xgb
from catboost import CatBoostClassifier

In [181]:
pca=PCA(n_components=45)
x_train_pca=pca.fit_transform(x_train_tr)
x_test_pca=pca.transform(x_test_tr)

In [187]:
tree=DecisionTreeClassifier()
tree.fit(x_train_pca,y_train_label)
y_pred_tr=tree.predict(x_test_pca)
accuracy_score(y_test_label,y_pred_tr)

0.6108

In [192]:

xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_clf.fit(x_train_pca,y_train_label)
y_pred_xgb = xgb_clf.predict(x_test_pca)

# Evaluate the model
accuracy = accuracy_score(y_test_label,y_pred_xgb)
print(f"XGBoost Accuracy: {accuracy:.2f}")
print(classification_report(y_test_label,y_pred_xgb))

XGBoost Accuracy: 0.72
              precision    recall  f1-score   support

           0       0.66      0.70      0.68       518
           1       0.75      0.72      0.73       822
           2       0.74      0.74      0.74      1160

    accuracy                           0.72      2500
   macro avg       0.71      0.72      0.72      2500
weighted avg       0.72      0.72      0.72      2500



In [195]:
gb=GaussianNB()
gb.fit(x_train_pca,y_train_label)
y_pred_gb=gb.predict(x_test_pca)
accuracy_score(y_test_label,y_pred_gb)
print(classification_report(y_test_label,y_pred_gb))

              precision    recall  f1-score   support

           0       0.60      0.79      0.68       518
           1       0.69      0.73      0.71       822
           2       0.77      0.64      0.70      1160

    accuracy                           0.70      2500
   macro avg       0.69      0.72      0.70      2500
weighted avg       0.71      0.70      0.70      2500



In [194]:

knn_clf = KNeighborsClassifier()
knn_clf.fit(x_train_pca,y_train_label)
y_pred_knn = knn_clf.predict(x_test_pca)

# Evaluate the model
accuracy1 = accuracy_score(y_test_label,y_pred_knn)
print(f"KNN Accuracy: {accuracy1:.2f}")
print(classification_report(y_test_label,y_pred_knn))

KNN Accuracy: 0.63
              precision    recall  f1-score   support

           0       0.54      0.60      0.57       518
           1       0.66      0.66      0.66       822
           2       0.67      0.63      0.65      1160

    accuracy                           0.63      2500
   macro avg       0.62      0.63      0.62      2500
weighted avg       0.64      0.63      0.64      2500



In [212]:

rf_clf = RandomForestClassifier()
rf_clf.fit(x_train_pca,y_train_label)
y_pred_rf = rf_clf.predict(x_test_pca)

# Evaluate the model
accuracy2 = accuracy_score(y_test_label,y_pred_rf)
print(f"Random Forest Accuracy: {accuracy2:.2f}")
print(classification_report(y_test_label,y_pred_rf))

Random Forest Accuracy: 0.72
              precision    recall  f1-score   support

           0       0.64      0.71      0.67       518
           1       0.74      0.73      0.73       822
           2       0.75      0.71      0.73      1160

    accuracy                           0.72      2500
   macro avg       0.71      0.72      0.71      2500
weighted avg       0.72      0.72      0.72      2500



In [203]:

lr_clf = LogisticRegression()
lr_clf.fit(x_train_pca,y_train_label)
y_pred_lr = lr_clf.predict(x_test_pca)

# Evaluate the model
accuracy = accuracy_score(y_test_label,y_pred_lr)
print(f"Logistic Regression Accuracy: {accuracy:.2f}")
print(classification_report(y_test_label,y_pred_lr))

Logistic Regression Accuracy: 0.71
              precision    recall  f1-score   support

           0       0.64      0.76      0.70       518
           1       0.71      0.69      0.70       822
           2       0.75      0.70      0.72      1160

    accuracy                           0.71      2500
   macro avg       0.70      0.72      0.71      2500
weighted avg       0.72      0.71      0.71      2500



In [204]:

gb_clf = GradientBoostingClassifier()
gb_clf.fit(x_train_pca,y_train_label)
y_pred_gb = gb_clf.predict(x_test_pca)

# Evaluate the model
accuracy = accuracy_score(y_test_label,y_pred_lr)
print(f"Gradiant Boosting Accuracy: {accuracy:.2f}")
print(classification_report(y_test_label,y_pred_gb))

Gradiant Boosting Accuracy: 0.71
              precision    recall  f1-score   support

           0       0.63      0.75      0.68       518
           1       0.75      0.73      0.74       822
           2       0.76      0.72      0.74      1160

    accuracy                           0.73      2500
   macro avg       0.72      0.73      0.72      2500
weighted avg       0.73      0.73      0.73      2500



In [207]:
estmator=[('lr1',lr_clf),('tree1',tree),('xgb',xgb_clf),('gb1',gb),('rf',rf_clf)]

In [209]:
vc=VotingClassifier(estimators=estmator,voting='soft',n_jobs=-1)
vc.fit(x_train_pca,y_train_label)
y_pred_vc = vc.predict(x_test_pca)

# Evaluate the model
accuracy = accuracy_score(y_test_label,y_pred_vc)
print(f"Voting Classifier Accuracy: {accuracy:.2f}")
print(classification_report(y_test_label,y_pred_vc))

Voting Classifier Accuracy: 0.72
              precision    recall  f1-score   support

           0       0.64      0.72      0.68       518
           1       0.73      0.73      0.73       822
           2       0.75      0.71      0.73      1160

    accuracy                           0.72      2500
   macro avg       0.71      0.72      0.71      2500
weighted avg       0.72      0.72      0.72      2500



In [210]:
sc=StackingClassifier(estimators=estmator,final_estimator=xgb_clf,n_jobs=-1)
sc.fit(x_train_pca,y_train_label)
y_pred_sc = sc.predict(x_test_pca)

# Evaluate the model
accuracy = accuracy_score(y_test_label,y_pred_sc)
print(f"Stacking Classifier Accuracy: {accuracy:.2f}")
print(classification_report(y_test_label,y_pred_sc))

Stacking Classifier Accuracy: 0.73
              precision    recall  f1-score   support

           0       0.65      0.73      0.69       518
           1       0.75      0.71      0.73       822
           2       0.75      0.74      0.74      1160

    accuracy                           0.73      2500
   macro avg       0.72      0.73      0.72      2500
weighted avg       0.73      0.73      0.73      2500



In [238]:
cat_clf = CatBoostClassifier(iterations=200, learning_rate=0.1, depth=6, verbose=0)

cat_clf.fit(x_train_pca,y_train_label)
y_pred_cat = cat_clf.predict(x_test_pca)

accuracy = accuracy_score(y_test_label, y_pred_cat)
print(f"CatBoost Accuracy: {accuracy:.2f}")
print(classification_report(y_test_label,y_pred_cat))

CatBoost Accuracy: 0.74
              precision    recall  f1-score   support

           0       0.65      0.79      0.71       518
           1       0.75      0.74      0.75       822
           2       0.78      0.71      0.75      1160

    accuracy                           0.74      2500
   macro avg       0.73      0.75      0.73      2500
weighted avg       0.74      0.74      0.74      2500



In [235]:
pipe=make_pipeline(encode,scale,transform,cat_clf)

In [236]:
pipe.fit(x_train,y_train_label)
y_pred_cat1=pipe.predict(x_test)
accuracy = accuracy_score(y_test_label, y_pred_cat1)
print(f"CatBoost Accuracy: {accuracy:.2f}")
print(classification_report(y_test_label,y_pred_cat1))

CatBoost Accuracy: 0.76
              precision    recall  f1-score   support

           0       0.65      0.79      0.71       518
           1       0.79      0.77      0.78       822
           2       0.80      0.74      0.77      1160

    accuracy                           0.76      2500
   macro avg       0.75      0.76      0.75      2500
weighted avg       0.76      0.76      0.76      2500



In [241]:
import joblib

joblib.dump(pipe, 'credit_score_model.joblib')


['credit_score_model.joblib']